In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed
from keras.layers import Flatten,LSTM
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

# 랜덤시드 고정시키기

np.random.seed(3)

Using TensorFlow backend.


In [2]:
# 데이터 생성 - 훈련 데이터 각 2000개, 검증 데이터 각 300개, 테스트 데이터 각 300개

# 300*300을 50*50으로 줄여서 학습시키도록 한다. 
# 훈련 데이터 수가 클래스당 2000개이므로 배치 크기를 5로 지정하면 400번 배치를 수행해 한 에포크 돌아감

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('train',target_size=(50, 50),batch_size=4,class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory('val',target_size=(50, 50),batch_size=4,class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('test',target_size=(50,50),batch_size=4, shuffle=False, class_mode='categorical')

Found 100 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [3]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

# Compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 36866     
Total params: 37,762
Trainable params: 37,762
Non-trainable params: 0
_________________________________________________________________


In [6]:
# 모델 학습시키기

# steps_per_epoch: 총 6000개의 데이터이고 batch_size=5이므로 1200스텝
# validation_steps: 총 900개의 검증 샘플이 있고 batch_size=5이므로 180스텝

hist = model.fit_generator(train_generator,epochs=5, validation_data = val_generator)

Epoch 1/5
25/25 [==============================] - 0s 17ms/step - loss: 0.0200 - acc: 1.0000 - val_loss: 1.9215 - val_acc: 0.5000
Epoch 2/5
25/25 [==============================] - 0s 19ms/step - loss: 0.0151 - acc: 1.0000 - val_loss: 0.8384 - val_acc: 0.2000
Epoch 3/5
25/25 [==============================] - 0s 19ms/step - loss: 0.0328 - acc: 0.9900 - val_loss: 3.5526 - val_acc: 0.4000
Epoch 4/5
25/25 [==============================] - 0s 16ms/step - loss: 0.0360 - acc: 0.9800 - val_loss: 1.1620 - val_acc: 0.2000
Epoch 5/5
25/25 [==============================] - 0s 16ms/step - loss: 0.0254 - acc: 0.9900 - val_loss: 1.9187 - val_acc: 0.4000


In [7]:
# 모델 평가하기 (훈련 데이터와 검증 데이터로 학습 후 맘에 들때까지 하이퍼 파리미터 (가중치 등) 조정하여 테스트 데이터로 평가)

print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=2)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
acc: 62.50%


In [8]:
# 모델 사용하기 (테스트 데이터에 대해 예측하기 (또는 그냥 예측하고 싶은 데이터를 제너레이터로 만들어서 넣으면 됨))

print("-- Predict --")
output = model.predict_generator(test_generator)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)
print()
print('\n'.join(test_generator.filenames))

-- Predict --
{'o': 0, 'x': 1}
[[0.989 0.011]
 [0.979 0.021]
 [0.990 0.010]
 [0.996 0.004]
 [0.997 0.003]
 [0.966 0.034]
 [0.977 0.023]
 [0.956 0.044]
 [0.965 0.035]
 [0.984 0.016]]

o\20200214_DWG_GEN_1_HL.mp4_20200730_212018.664.jpg
o\20200214_DWG_GEN_1_HL.mp4_20200730_212019.165.jpg
o\20200214_DWG_GEN_1_HL.mp4_20200730_212019.665.jpg
o\20200214_DWG_GEN_1_HL.mp4_20200730_212020.167.jpg
o\20200214_DWG_GEN_1_HL.mp4_20200730_212020.666.jpg
x\20200214_DWG_GEN_1_HL.mp4_20200730_212016.162.jpg
x\20200214_DWG_GEN_1_HL.mp4_20200730_212016.663.jpg
x\20200214_DWG_GEN_1_HL.mp4_20200730_212017.163.jpg
x\20200214_DWG_GEN_1_HL.mp4_20200730_212017.663.jpg
x\20200214_DWG_GEN_1_HL.mp4_20200730_212018.163.jpg
